In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_Vale = yf.download(tickers='VALE3.SA', start='2022-01-01', end='2025-08-01', multi_level_index=False)
df_Vale = df_Vale[['Close', 'Open', 'Volume']]
df_Vale.reset_index(inplace=True)
df_Vale['Date'] = pd.to_datetime(df_Vale['Date'])
df_Vale.set_index('Date', inplace=True)
df_Vale.rename(columns={
    'Close': 'Close_VALE3',
    'Open': 'Open_VALE3',
    'Volume': 'Volume_VALE3'
}, inplace=True)
df_Vale

[*********************100%***********************]  1 of 1 completed


,Close_VALE3,Open_VALE3,Volume_VALE3
Date,,,
2022-01-03,57.766422,58.507017,18557200
2022-01-04,57.085064,58.144115,18178700
2022-01-05,57.625698,57.299840,22039000
2022-01-06,58.788429,58.240391,22044100
2022-01-07,62.209984,59.543843,35213100
...,...,...,...
2025-07-25,55.700001,56.180000,21918900
2025-07-28,55.160000,54.939999,19175100
2025-07-29,54.820000,55.549999,14648700


In [3]:
df_ferro = pd.read_csv("Dados Históricos - Minério de ferro refinado 62% Fe CFR Futuros.csv")
df_ferro.reset_index(inplace=True)
df_ferro['Data'] = pd.to_datetime(df_ferro['Data'], dayfirst=True)
df_ferro.set_index('Data', inplace=True)
df_ferro = df_ferro[['Último','Abertura', 'Var%']]
# Invertendo a ordem das datas:
df_ferro = df_ferro.sort_index(ascending=True)
df_ferro.rename(columns={
    'Último': 'Close_Ferro',
    'Abertura': 'Open_Ferro',
    'Var%': 'Variacao_ferro'
}, inplace=True) 
df_ferro

,Close_Ferro,Open_Ferro,Variacao_ferro
Data,,,
2022-01-03,"120,40","120,40","7,02%"
2022-01-04,"120,91","120,91","0,42%"
2022-01-05,"124,14","124,14","2,67%"
2022-01-06,"125,94","125,94","1,45%"
2022-01-07,"126,21","126,21","0,21%"
...,...,...,...
2025-07-25,"98,55","98,55","-0,03%"
2025-07-28,"98,67","98,67","0,12%"
2025-07-29,"98,98","98,98","0,31%"


In [4]:
# Agora vamos unir os dataframes:

df = pd.merge(df_Vale, df_ferro, left_index=True, right_index=True, suffixes=('_VALE3', '_Ferro'))

# Variação percentual do preço de fechamento da VALE3:
df['Variação_VALE3'] = df_Vale['Close_VALE3'].pct_change() * 100
# Trocando nan por 0%:
df['Variação_VALE3'].fillna(0, inplace=True)

# Vamos transformar as colunas Close_Minerio, Open_Minerio e Variação_Minerio em float:
df['Close_Ferro'] = df['Close_Ferro'].str.replace(',', '.', regex=False).astype(float)
df['Open_Ferro'] = df['Open_Ferro'].str.replace(',', '.', regex=False).astype(float)
df['Variacao_ferro'] = df['Variacao_ferro'].str.replace('%', '', regex=False).str.replace(',', '.', regex=False).astype(float)

df

,Close_VALE3,Open_VALE3,Volume_VALE3,Close_Ferro,Open_Ferro,Variacao_ferro,Variação_VALE3
2022-01-03,57.766422,58.507017,18557200,120.40,120.40,7.02,0.000000
2022-01-04,57.085064,58.144115,18178700,120.91,120.91,0.42,-1.179506
2022-01-05,57.625698,57.299840,22039000,124.14,124.14,2.67,0.947068
2022-01-06,58.788429,58.240391,22044100,125.94,125.94,1.45,2.017730
2022-01-07,62.209984,59.543843,35213100,126.21,126.21,0.21,5.820116
...,...,...,...,...,...,...,...
2025-07-25,55.700001,56.180000,21918900,98.55,98.55,-0.03,-1.468243
2025-07-28,55.160000,54.939999,19175100,98.67,98.67,0.12,-0.969481
2025-07-29,54.820000,55.549999,14648700,98.98,98.98,0.31,-0.616389
2025-07-30,53.840000,54.320000,35607800,99.07,99.07,0.09,-1.787668


In [5]:
df.reset_index(inplace=True)
df['index'] = pd.to_datetime(df['index'], dayfirst=True)
df.rename(columns={'index': 'Data'}, inplace=True)
df.set_index('Data', inplace=True)
df

,Close_VALE3,Open_VALE3,Volume_VALE3,Close_Ferro,Open_Ferro,Variacao_ferro,Variação_VALE3
Data,,,,,,,
2022-01-03,57.766422,58.507017,18557200,120.40,120.40,7.02,0.000000
2022-01-04,57.085064,58.144115,18178700,120.91,120.91,0.42,-1.179506
2022-01-05,57.625698,57.299840,22039000,124.14,124.14,2.67,0.947068
2022-01-06,58.788429,58.240391,22044100,125.94,125.94,1.45,2.017730
2022-01-07,62.209984,59.543843,35213100,126.21,126.21,0.21,5.820116
...,...,...,...,...,...,...,...
2025-07-25,55.700001,56.180000,21918900,98.55,98.55,-0.03,-1.468243
2025-07-28,55.160000,54.939999,19175100,98.67,98.67,0.12,-0.969481
2025-07-29,54.820000,55.549999,14648700,98.98,98.98,0.31,-0.616389


# Adicionando features no nosso DataFrame:

* Médias Móveis:

In [6]:
janelas = [7,20,200]

for janela in janelas:
    df[f'MM_{janela}D'] = df['Close_VALE3'].rolling(window=janela, min_periods=1).mean()
    

In [8]:
df[['MM_7D', 'MM_20D', 'MM_200D']].isnull().sum()

MM_7D      0
MM_20D     0
MM_200D    0
dtype: int64

* Retorno diário e retorno de 7 dias anteriores:

In [9]:
retornos = [1,7,21]

for retorno in retornos:
    df[f'Retorno_{retorno}D'] = df['Close_VALE3'].pct_change(retorno) * 100
    df[f'Retorno_{retorno}D'].fillna(0, inplace=True)

In [10]:
df[['Retorno_1D', 'Retorno_7D', 'Retorno_21D']]

,Retorno_1D,Retorno_7D,Retorno_21D
Data,,,
2022-01-03,0.000000,0.000000,0.000000
2022-01-04,-1.179506,0.000000,0.000000
2022-01-05,0.947068,0.000000,0.000000
2022-01-06,2.017730,0.000000,0.000000
2022-01-07,5.820116,0.000000,0.000000
...,...,...,...
2025-07-25,-1.468243,2.389704,10.340732
2025-07-28,-0.969481,1.583795,6.076923
2025-07-29,-0.616389,0.476537,3.688294


In [11]:
def Rolling_Sharpe_Ratio(returns, window=21, risk_free_rate=0.0, trading_days=252):
    # Converter taxa livre de risco para diária:
    rf_daily = (1 + risk_free_rate)**(1/trading_days) - 1

    # Calcular retornos excedentes:
    excess_return = returns - rf_daily

    # Média móvel dos retornos excedentes:
    mean_returns = excess_return.rolling(window).mean()

    # Desvio padrão móvel dos retornos excedentes:
    std_returns = excess_return.rolling(window).std()

    # Sharpe Ratio anualizado:
    sharpe_ratio = mean_returns / std_returns * np.sqrt(trading_days)

    return sharpe_ratio

# Calculando o sharpe ratio de 21 dias( 21 dias = 1 mês de trading):
df['Sharpe_21D'] = Rolling_Sharpe_Ratio(
    returns=df['Retorno_1D'],
    window=21,
    risk_free_rate= 0.15, # 15% ao ano (SELIC atual)
    trading_days=252
)

In [12]:
# Tratando valores NaN do sharpe de 21 dias:
df['Sharpe_21D'].fillna(df['Sharpe_21D'].rolling(window=3, min_periods=1).mean(), inplace=True)
df['Sharpe_21D'].isnull().sum()

20

In [13]:
# Tratando valores NaN do sharpe de 21 dias:
df['Sharpe_21D'].fillna(0, inplace=True)
df['Sharpe_21D'].isnull().sum()

0

In [18]:
df.loc[df['Sharpe_21D'] == 0.000000, 'Sharpe_21D'] = df['Sharpe_21D'].rolling(window=7, min_periods=1).mean()
df['Sharpe_21D'].isnull().sum()

0

In [19]:
df[['Sharpe_21D']]

,Sharpe_21D
Data,
2022-01-03,0.000000
2022-01-04,0.000000
2022-01-05,0.000000
2022-01-06,0.000000
2022-01-07,0.000000
...,...
2025-07-25,4.435949
2025-07-28,2.817116
2025-07-29,1.793258
